# import

In [1]:
import os
import io
import cv2
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm

# load csv

In [2]:
dataset_dir = 'dataset/'

train_image_path = dataset_dir+'train_image_level.csv'
train_study_path = dataset_dir+'train_study_level.csv'

submission_path = dataset_dir+'sample_submission.csv'

In [3]:
train_image_df = pd.read_csv(train_image_path)
train_study_df = pd.read_csv(train_study_path)

submission_df = pd.read_csv(submission_path)

In [4]:
train_image_df = pd.read_csv(train_image_path)
train_study_df = pd.read_csv(train_study_path)

In [5]:
train_study_df['StudyInstanceUID'] = train_study_df['id'].apply(lambda x: x.replace('_study', ''))
train_image_df['id'] = train_image_df['id'].apply(lambda x: x.replace('_image', ''))
del train_study_df['id']
train_df = train_image_df.merge(train_study_df, on='StudyInstanceUID')
del train_study_df
del train_image_df
train_df.head()

,id,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,0,1,0,0
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,1,0,0,0
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0,1,0,0
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,0,0,0,1
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,0,1,0,0


In [6]:
train_df = train_df.dropna(axis=0)
train_df

,id,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,0,1,0,0
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0,1,0,0
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,0,0,0,1
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,0,1,0,0
5,0022227f5adf,"[{'x': 1857.2065, 'y': 508.30565, 'width': 376...",opacity 1 1857.2065 508.30565 2233.23384 907.8...,84543edc24c2,0,0,1,0
...,...,...,...,...,...,...,...,...
6327,ffbeafe30b77,"[{'x': 1472.09879, 'y': 1182.64415, 'width': 3...",opacity 1 1472.09879 1182.64415 1773.17398 146...,f75774536369,0,1,0,0
6328,ffcc16bbf428,"[{'x': 2529.03977, 'y': 347.67053, 'width': 58...",opacity 1 2529.03977 347.67053 3113.65526 1484...,5d786634f354,0,1,0,0
6331,ffd9b6cf2961,"[{'x': 2197.38566, 'y': 841.07361, 'width': 31...",opacity 1 2197.38566 841.07361 2513.80265 1292...,7eed9af03814,0,1,0,0
6332,ffdc682f7680,"[{'x': 2729.27083, 'y': 332.26044, 'width': 14...",opacity 1 2729.27083 332.26044 4225.52099 2936...,a0cb0b96fb3d,0,1,0,0


# resize function

In [7]:
train_df['boxes'] = train_df['boxes'].str.replace(r'\'','"')

for index, data in tqdm(train_df.iterrows()):
    if not glob('dataset/train/'+data['StudyInstanceUID']+'/*/'+data['id']+'.jpg'):
        print(str(index)+' not file')
        train_df.drop(index)
        continue
    else:
        path = glob('dataset/train/'+data['StudyInstanceUID']+'/*/'+data['id']+'.jpg')[0]
        
    imageToPredict = cv2.imread(path, 3)
    
    y = imageToPredict.shape[0]
    x = imageToPredict.shape[1]

    targetSize = 640
    
    if(x==targetSize and y==targetSize):
        continue
    
    x_scale = targetSize / x
    y_scale = targetSize / y
    img = cv2.resize(imageToPredict, (targetSize, targetSize))
    img = np.array(img)
    
    new_dict = []
    boxes = json.loads(data['boxes'])
    
    for box in boxes:
        new_box = {}
        new_box['xmin'] = int(np.round(box['x'] * x_scale))
        new_box['ymin'] = int(np.round(box['y'] * y_scale))
        new_box['xmax'] = int(np.round((box['x']+box['width']) * x_scale))
        new_box['ymax'] = int(np.round((box['y']+box['height']) * y_scale))
        new_dict.append(new_box)
                   
    train_df['boxes'][index] = json.dumps(new_dict)
    cv2.imwrite(path, img)

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
4294it [04:35, 15.57it/s]


# save csv

In [11]:
train_df = train_df.drop(['label'],axis=1)
train_df.head()

,id,boxes,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,000a312787f2,"[{""xmin"": 119, ""ymin"": 107, ""xmax"": 273, ""ymax...",5776db0cec75,0,1,0,0
2,0012ff7358bc,"[{""xmin"": 142, ""ymin"": 50, ""xmax"": 324, ""ymax""...",9d514ce429a7,0,1,0,0
3,001398f4ff4f,"[{""xmin"": 408, ""ymin"": 397, ""xmax"": 550, ""ymax...",28dddc8559b2,0,0,0,1
4,001bd15d1891,"[{""xmin"": 117, ""ymin"": 240, ""xmax"": 251, ""ymax...",dfd9fdd85a3e,0,1,0,0
5,0022227f5adf,"[{""xmin"": 390, ""ymin"": 128, ""xmax"": 469, ""ymax...",84543edc24c2,0,0,1,0


In [12]:
train_df.to_csv('dataset/mod_train_df.csv',index=False)